Copyright (c) 2020 Martin Holle. Alle Rechte vorbehalten. Lizensiert unter der MIT-Lizenz.

# Covid-19 Statistics Aachen: Datenaufbereitung

Die rohen Fallzahlen werden zunächst aus dem im ersten Schritt erzeugten Excel-Sheet eingelesen und dann aufbereitet. Anschließend wird eine Reihe von Kennwerten berechnet. Die Ergebnisse werden in eine zweite Excel-Datei exportiert; die zwischengespeicherten Daten werden im folgenden Schritt zur Erstellung von Diagrammen verwendet.

## Benötigte Imports und Initialisierungen

In [1]:
import pandas as pd

import logging
import configparser

# Konfiguration einlesen
config = configparser.ConfigParser(interpolation=None)
config.read('config.ini')

# Konfiguration des Loggings
fhandler = logging.FileHandler(filename=config['Logging']['LogFileName'], mode='a')
formatter = logging.Formatter(config['Logging']['LogFormat'])
fhandler.setFormatter(formatter)

# Logger instanzieren
log = logging.getLogger("aufbereitung")
log.addHandler(fhandler)
log.setLevel(logging.INFO)

## Einlesen der Excel-Datei und Anzeige der eingelesenen Daten

- Datei: Siehe `config.ini`
- Seite: Siehe `config.ini`
- Einzulesende Spalten: 
  - **A**: Datum im Format 'DD.MM.'
  - **B**: Akkumulierte Anzahl der Infektionen für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **D**: Akkumulierte Anzahl der Infektionen für die Stadt Aachen als Integerzahl
  - **F**: Akkumulierte Anzahl der Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl 
  - **G**: Akkumulierte Anzahl der Genesenen für gesamte Städteregion (inkl. Aachen) als Integerzahl
- Spaltentypen: Spalte A als Datum interpretieren
- Die erste Zeile enthält die Spalten-Header
- Label der Spalten explizit setzen

In [2]:
col_names = ['Datum', 'Uhrzeit', 'Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]

try:
    c19_cases = pd.read_excel(config['Rohdaten']['FileName'], 
                              sheet_name=config['Rohdaten']['SheetName'],
                              usecols='A,C,D,E,F,G', 
                              index_col=0,
                              engine='openpyxl',
                              skiprows=[])

    if len(c19_cases.index) > 0:
        log.info('{0} data records read from file {1}'.format(len(c19_cases.index), config['Rohdaten']['FileName']))        
        print(c19_cases.tail(14))
    else:
        log.error('No data records read from file {0}'.format(config['Rohdaten']['FileName']))        

except FileNotFoundError as err: 
    log.error('Error during pd.read_excel(): {0}'.format(err))

            Summe  Summe Aachen  Summe Todesfälle  Summe genesen  Akute Fälle
Datum                                                                        
2020-12-28  14012          5982               263          11950         1799
2020-12-29  14078          6014               265          12189         1624
2020-12-30  14248          6077               272          12435         1541
2020-12-31  14395          6133               276          12460         1659
2021-01-04  14721          6268               292          12848         1581
2021-01-05  14723          6259               301          12949         1473
2021-01-06  14832          6295               310          13203         1319
2021-01-07  15028          6375               319          13437         1272
2021-01-08  15211          6451               325          13473         1413
2021-01-11  15460          6569               335          13759         1366
2021-01-12  15469          6577               340          14015

## Qualitätskontrolle
Die Summe der Fälle (für die Städteregion) abzüglich der Summe der Todesfälle und abzüglich der Summe der Genesenen muss gleich der Anzahl der akuten Fälle sein. Die resultierende Liste ist leer, wenn alle Zahlen korrekt sind.

In [3]:
error_cases = c19_cases[c19_cases['Summe'] - c19_cases['Summe Todesfälle'] - c19_cases['Summe genesen'] != c19_cases['Akute Fälle']]

if len(error_cases.index) > 0:
    log.error('{0} inconsistence data records detected:'.format(len(error_cases.index)))
    for i in range(0, len(error_cases.index)):
        log.error('  Date: {0}, Sum: {1}, Deaths: {2}, Recovered: {3}, Active: {4} -> should be: {5}'.format(
            error_cases.index[i], 
            error_cases.values[i][0],
            error_cases.values[i][2],
            error_cases.values[i][3],
            error_cases.values[i][4],
            error_cases.values[i][0] - error_cases.values[i][2] - error_cases.values[i][3]))
else:
    log.info('Data consistency check passed')        

## Behandlung fehlender Werte

### Spalte 'Summe Todesfälle'

Am 17.03.2020 wurde das erste Mal in der offiziellen Verlautbarung ein Todesfall erwähnt; für alle Tage vorher wird dieser Spaltenwert zu Null angenommen. Wenn an einem Tag nach dem 17.03. kein Todesfall in der Meldung verzeichnet ist, wird dieser Spaltenwert jeweils fortgeschrieben mit dem letzten offiziell gemeldeten; dadurch kann es zu Sprüngen in der Statistik kommen.

### Spalte 'Summe genesen'

Das erste Mal wurde in der offiziellen Verlautbarung am 10.03.2020 die Zahl der wieder Genesenen erwähnt, bis zum 31.03. erfolgten nur einzelne Meldungen zu sog. "Freitestungen". Erst danach wird die Zahl der Genesenen regelmäßig in den Meldungen der Krisenstäbe erwähnt. Für alle Tage, für die keine Meldung existiert, wurde dieser Spaltenwert jeweils fortgeschrieben mit dem letzten offiziell gemeldeten; dadurch kann es zu Sprüngen in der Statistik kommen.

### Hinzufügen von Tagen, für die keine Fallzahlen vorliegen

Ab dem 01.05.2020 sind die Krisenstäbe von Städteregion und der Stadt Aachen dazu übergegangen, neue Fallzahlen nur noch an normalen Wochentagen zu veröffentlichen; ab Juni wurden nur noch 3x pro Woche die Fallzahlen bekannt gegeben. Damit enthalten die aus dem Excelsheet eingelesenen Zeitreihen tageweise Lücken, insbesondere für die Wochenenden. Die größte Lücke trat Weihnachten auf, als 4 Tage hintereinander keine neuen Zahlen veröffentlicht wurden. Diese Lücken werden in den folgenden Schritten aufgefüllt, um die Berechnung gleitender Mittelwerte und Summen später zu vereinfachen.

#### Methode: Lineare Interpolation
- Im ersten Schritt (`reindex`) die fehlenden Werte mit `NaN` auffüllen
- Im zweiten Schritt per `interpolate` (`method='linear'`) die `NaN` durch linear interpolierte Werte ersetzen
- Im dritten Schritt die neu berechnete Spalte für `Akute Fälle` hinzufügen

_Begründung:_

Insbesondere bei der Berechnung der Inzidenz ergeben sich beim vorher verwendeten einfachen Fortschreiben von Werten per `bfill` oder `ffill` inakzeptable Sprünge im Kurvenverlauf. Realistischer erscheint die Annahme, dass die Werte zwischen den bekannten Stützwerte annähernd _linear_ verlaufen.

In [4]:
# Hinzufügen von Tagen, für die keine Fallzahlen vorliegen und fehlende Werte ergänzen

# Dataframe neu indizieren: Fehlende Spaltenwerte beim Reindizieren zunächst mit NaN auffüllen
c19_cases = c19_cases.reindex(pd.date_range(c19_cases.index[0], c19_cases.index[-1]))

# NaN-Spaltenwerte durch lineare Interpolation auffüllen 
c19_cases = c19_cases[['Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen']].interpolate()

c19_cases

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen
2020-03-01,9.0,4.0,0.0,0.0
2020-03-02,10.0,5.0,0.0,0.0
2020-03-03,13.0,5.0,0.0,0.0
2020-03-04,24.0,8.0,0.0,0.0
2020-03-05,33.0,10.0,0.0,0.0
...,...,...,...,...
2021-01-11,15460.0,6569.0,335.0,13759.0
2021-01-12,15469.0,6577.0,340.0,14015.0
2021-01-13,15548.0,6609.0,347.0,14173.0
2021-01-14,15657.0,6651.0,347.0,14233.0


#### Korrektur der Spaltenformate
Die Fallzahlen wurden wegen fehlender Werte teilweise als Fließkommazahlen (NaN erfordert ein Float zu Darstellung) importiert, es handelt sich aber in allen Spalten um Ganzzahlen.

In [5]:
# Explizite Umwandlung aller Spalten (außer Index) nach Integer
c19_cases = c19_cases.astype('int32')

#### Neuberechnung der Spalte `Akute Fälle`

In [6]:
c19_cases['Akute Fälle'] = c19_cases['Summe'] - c19_cases['Summe Todesfälle'] - c19_cases['Summe genesen']
                                     
c19_cases

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
2020-03-01,9,4,0,0,9
2020-03-02,10,5,0,0,10
2020-03-03,13,5,0,0,13
2020-03-04,24,8,0,0,24
2020-03-05,33,10,0,0,33
...,...,...,...,...,...
2021-01-11,15460,6569,335,13759,1366
2021-01-12,15469,6577,340,14015,1114
2021-01-13,15548,6609,347,14173,1028
2021-01-14,15657,6651,347,14233,1077


## Berechnung zusätzlicher Spalten

- Absolutes und prozentuales Wachstum (oder Rückgang) der aktiven Fälle berechnen
- Anzahl der Neuinfektionen pro Tag (für Stadt Aachen und Städteregion) berechnen
- Anzahl der neuen Todesfälle pro Tag (für Städteregion) berechnen
- Anzahl der neu Genesenen pro Tag (für Städteregion) berechnen
- Gleitendes Mittel (über `Glaettung.Intervall` Tage) für die obigen Kennwerte berechnen
- Summen der Fälle bezogen auf 100.000 Einwohner (für Stadt Aachen und Städteregion) berechnen
- Summe der Neuinfektionen der letzten 7 Tage pro 100.000 Einwohner („7-Tage-Inzidenz“ für Stadt Aachen und Städteregion) berechnen

In [7]:
Einwohner_StädteRegion = int(config['Kennzahlen']['Einwohner.Region'])
Einwohner_StadtAachen = int(config['Kennzahlen']['Einwohner.Aachen'])
Glättungsintervall = int(config['Kennzahlen']['Glaettung.Intervall'])

def change_per_cent(x):
    """
    Evaluate and return the percentage change of the active cases per day by using apply().

    Evaluates difference between first and last array entry, which represents the 'Rolling Window'. 
    Divide by first value and number of days in between.
    
    Parameters
    ----------
    x : []
        Array with case numbers 
    
    Returns
    -------
    float
        Percent change of active cases per day
    """
    if x.size == 1:
        return 0
    else:
        return (x[-1] - x[0]) / x[0] / (x.size - 1) * 100

def diff(x):
    """
    Evaluate new cases / recovered cases / deaths per day by using apply().

    Evaluate difference between first and last array entry, which represents the 'Rolling Window'.
    Divide by number of days in between.

    Parameters
    ----------
    x : []
        Array with case numbers 
    
    Returns
    -------
    float
        Change of cases per day
    """
    if x.size == 1:
        return 0
    else:
        return (x[-1] - x[0]) / (x.size - 1)
    
# Aktive Fälle
c19_cases['Änderung Akute Fälle'] = c19_cases['Akute Fälle'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Änderung Akute Fälle (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Änderung Akute Fälle'].rolling(Glättungsintervall, center=True).mean()
c19_cases['Änderung Akute Fälle [%]'] = c19_cases['Akute Fälle'].rolling(2, min_periods=1).apply(change_per_cent, raw=True)
c19_cases['Änderung Akute Fälle (MW/{0}T) [%]'.format(str(Glättungsintervall))] = c19_cases['Änderung Akute Fälle [%]'].rolling(Glättungsintervall, center=True).mean()

# Neuinfektionen
c19_cases['Neuinfektionen'] = c19_cases['Summe'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neuinfektionen Aachen'] = c19_cases['Summe Aachen'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neuinfektionen (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Neuinfektionen'].rolling(Glättungsintervall, center=True).mean()

# Neue Todesfälle
c19_cases['Neue Todesfälle'] = c19_cases['Summe Todesfälle'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neue Todesfälle (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Neue Todesfälle'].rolling(Glättungsintervall, center=True).mean()

# Neue Genesene
c19_cases['Neue Genesene'] = c19_cases['Summe genesen'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neue Genesene (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Neue Genesene'].rolling(Glättungsintervall, center=True).mean()

# Fälle pro 100.000 Einwohner in den letzten 7 Tagen
c19_cases['Summe pro 100000'] = c19_cases['Summe'] / Einwohner_StädteRegion * 100000
c19_cases['Summe Aachen pro 100000'] = c19_cases['Summe Aachen'] / Einwohner_StadtAachen * 100000
c19_cases['Neuinfektionen 7T/100000 Aachen'] = c19_cases['Neuinfektionen Aachen'].rolling(7, center=False).sum() / Einwohner_StadtAachen * 100000
c19_cases['Neuinfektionen 7T/100000'] = c19_cases['Neuinfektionen'].rolling(7, center=False).sum() / Einwohner_StädteRegion * 100000


## Zwischenergebnis in Excel-Datei speichern

In [8]:
try:
    c19_cases.to_excel(config['Kennzahlen']['FileName'],
                       engine='openpyxl',
                       sheet_name=config['Kennzahlen']['SheetName'],
                       index_label='Datum')
    log.info("{0} processed records written to excel file {1}".format(len(c19_cases.index), config['Kennzahlen']['FileName']))

except Exception as err:
    log.error('Error during c19_cases.to_excel(): {0}'.format(err))

##  Ausgabe der prozessierten Daten zur Kontrolle

In [9]:
c19_cases

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle,Änderung Akute Fälle,Änderung Akute Fälle (MW/7T),Änderung Akute Fälle [%],Änderung Akute Fälle (MW/7T) [%],Neuinfektionen,Neuinfektionen Aachen,Neuinfektionen (MW/7T),Neue Todesfälle,Neue Todesfälle (MW/7T),Neue Genesene,Neue Genesene (MW/7T),Summe pro 100000,Summe Aachen pro 100000,Neuinfektionen 7T/100000 Aachen,Neuinfektionen 7T/100000
2020-03-01,9,4,0,0,9,0,NaN,0.000000,NaN,0,0,NaN,0,NaN,0,NaN,1.615724,1.606684,NaN,NaN
2020-03-02,10,5,0,0,10,1,NaN,11.111111,NaN,1,1,NaN,0,NaN,0,NaN,1.795248,2.008355,NaN,NaN
2020-03-03,13,5,0,0,13,3,NaN,30.000000,NaN,3,0,NaN,0,NaN,0,NaN,2.333823,2.008355,NaN,NaN
2020-03-04,24,8,0,0,24,11,6.571429,84.615385,31.993749,11,3,6.571429,0,0.000000,0,0.000000,4.308596,3.213368,NaN,NaN
2020-03-05,33,10,0,0,33,9,6.857143,37.500000,32.513229,9,2,6.857143,0,0.000000,0,0.000000,5.924320,4.016710,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-11,15460,6569,335,13759,1366,-17,-27.857143,-1.229212,-1.954322,83,40,89.857143,4,4.000000,96,113.714286,2775.453929,2638.576478,120.902956,132.668852
2021-01-12,15469,6577,340,14015,1114,-252,-65.428571,-18.448023,-5.156132,9,8,76.285714,5,4.857143,256,136.857143,2777.069652,2641.789846,127.731362,133.925526
2021-01-13,15548,6609,347,14173,1028,-86,NaN,-7.719928,NaN,79,32,NaN,7,NaN,158,NaN,2791.252114,2654.643316,126.124679,128.539781
2021-01-14,15657,6651,347,14233,1077,49,NaN,4.766537,NaN,109,42,NaN,0,NaN,60,NaN,2810.820321,2671.513496,110.861183,112.921120
